### Recurrent Neural Network

In [ ]:
import torch

def default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')   
    if torch.backends.mps.is_available():
        return torch.device('mps')
    return torch.device('cpu')

device = default_device()

In [ ]:
import pandas_datareader as pdr
gs10 = pdr.get_data_fred('GS10')
gs10.head()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(gs10)
plt.show()

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
num = len(gs10)
x = torch.tensor(gs10['GS10'].to_list())
seq_len = 6
batch_size = 4

X_feature = torch.zeros((num - seq_len, seq_len))
Y_feature = torch.zeros(num - seq_len, 1)

for i in range(num - seq_len):
    X_feature[i] = x[i:i+seq_len]
    Y_feature[i] = x[i+seq_len]

train_loader = DataLoader(TensorDataset(X_feature[:num-seq_len], Y_feature[:num-seq_len]), 
                          batch_size=batch_size, shuffle=True)

train_loader.dataset[:batch_size]


In [ ]:
from tqdm import *

In [ ]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Model, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = torch.relu(self.linear1(x))
        x = self.linear2(x)
        return x

In [ ]:
imput_size = seq_len
output_size = 1
hidden_size = 10
lr = 0.01

model = Model(imput_size, hidden_size, output_size).to(device)
criterion = nn.MSELoss(reduction='none')
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


In [ ]:
n_epochs = 20
loss_history = []
for epoch in tqdm(range(n_epochs)):
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        loss.sum().backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        total_loss = 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            total_loss += loss.sum().item() / loss.numel()
        avg_loss = total_loss / len(train_loader)
        loss_history.append(avg_loss)
        print(f'epoch: {epoch+1}, Validation Loss: {avg_loss:.4f}')

In [ ]:
plt.plot(loss_history, label='loss')
plt.legend()
plt.show()

In [ ]:
preds = model(X_feature.to(device)).cpu().detach().numpy()
times = torch.arange(1, num + 1, dtype=torch.float32)

plt.plot(times[:num - seq_len], gs10['GS10'][:num-seq_len], label='GS10')
plt.plot(times[seq_len:], preds, label='prediction')
plt.legend()
plt.show()

### RNN

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

num = len(gs10)
x = torch.tensor(gs10['GS10'].to_list())
seq_len = 6
batch_size = 4

X_feature = torch.zeros((num - seq_len, seq_len))
Y_label = torch.zeros(num - seq_len, seq_len)

for i in range(seq_len):
    X_feature[:, i] = x[i:num-seq_len+i]
    Y_label[:, i] = x[i+1:num-seq_len+i+1]

train_loader = DataLoader(TensorDataset(X_feature[:num-seq_len].unsqueeze(2), Y_label[:num-seq_len]),
                          batch_size=batch_size, shuffle=True)

train_loader.dataset[:batch_size]

In [ ]:
from torch import nn
from tqdm import *

class RNNModel(nn.Module):
    def __init__(self, input_size, output_size, num_hiddens, n_layers):
        super(RNNModel, self).__init__()
        self.num_hiddens = num_hiddens
        self.n_layers = n_layers
        self.rnn = nn.RNN(input_size, num_hiddens, n_layers, batch_first = True)
        self.linear = nn.Linear(num_hiddens, output_size)
       
    def forward(self, X):
        batch_size = X.size(0)
        state = self.begin_state(batch_size)
        output, state = self.rnn(X, state)
        output = self.linear(torch.relu(output))
        return output, state

    def begin_state(self, batch_size=1):
        return  torch.zeros(self.n_layers, batch_size, self.num_hiddens)

# 定义超参数
input_size = 1
output_size = 1
num_hiddens = 10
n_layers = 1
lr = 0.01

# 建立模型
model = RNNModel(input_size, output_size, num_hiddens, n_layers)
criterion = nn.MSELoss(reduction='none')
trainer = torch.optim.Adam(model.parameters(), lr)

In [ ]:
num_epochs = 20
rnn_loss_history = []

for epoch in tqdm(range(num_epochs)):
    # 批量训练
    for X, Y in train_loader:
        trainer.zero_grad()
        y_pred, state = model(X)
        loss = criterion(y_pred.squeeze(), Y.squeeze())
        loss.sum().backward()
        trainer.step()
    # 输出损失
    with torch.no_grad():
        total_loss = 0
        for X, Y in train_loader:
            y_pred, state = model(X)
            loss = criterion(y_pred.squeeze(), Y.squeeze())
            total_loss += loss.sum()/loss.numel()
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch {epoch+1}: Validation loss = {avg_loss:.4f}')
        rnn_loss_history.append(avg_loss)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(loss_history, label='loss')
plt.plot(rnn_loss_history, label='RNN_loss')
plt.legend()
plt.show()

In [ ]:
rnn_preds,_ = model(X_feature.unsqueeze(2))
preds.squeeze()
time = torch.arange(1, num+1, dtype= torch.float32)  # 时间轴

plt.plot(time[:num-seq_len], gs10['GS10'].to_list()[seq_len:num], label='gs10')
plt.plot(time[:num-seq_len], preds, label='preds')
plt.plot(time[:num-seq_len], rnn_preds[:,seq_len-1].detach().numpy(), label='RNN_preds')
plt.legend()
plt.show()